In [1]:
import pandas as pd
import random
import numpy as np

In [2]:
def to_sequence(data, seq_size=10):
    x, y = [], []

    for i in range(len(data)-seq_size):
        window = data[i:(i+seq_size)].values
        after_window = data[i+seq_size]
        x.append(window)
        y.append(after_window)

    return np.array(x), np.array(y)

In [3]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
from sklearn.svm import SVR
from sklearn.metrics import *

vertical_MAE =[]
vertical_MSE=[]
vertical_RMSE =[]
vertical_MRE=[]
vertical_MMRE =[]
vertical_PRED=[]

for j in range(1,40):
  # Read csv file into a pandas dataframe
  df = pd.read_csv(f'/content/Patient_{j}.csv')
  print(j)

  for i in range(len(df)):
    if np.isnan(df.at[i,'SensorGLU']):
      X, y = to_sequence(df[:i-1].SensorGLU)
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
      # Create a SVR model
      model = SVR(kernel='linear',gamma='auto', C=50, epsilon = 0.1)
      # Train the model
      model.fit(X_train, y_train)
      # Make imputation
      df.at[i,'SensorGLU']= model.predict((df.loc[i-10:i-1, "SensorGLU"].values).reshape(1,10))


  # Model Assessing
  pred = model.predict(X_test)
  vertical_MAE =np.append(vertical_MAE , mean_absolute_error(y_test,pred))
  vertical_MSE =np.append(vertical_MSE , mean_squared_error(y_test,pred))
  vertical_RMSE =np.append(vertical_RMSE , np.sqrt(mean_squared_error(y_test,pred)))
  vertical_MRE = np.append(vertical_MRE , np.abs((y_test-pred)/y_test)*100)
  vertical_MMRE = np.append(vertical_MMRE , np.mean(np.abs((y_test-pred)/y_test)*100))
  Pred = lambda p: len(vertical_MRE[vertical_MRE <= p])/len(vertical_MRE)*100
  PRED = Pred(25)
  vertical_PRED =np.append(vertical_PRED , PRED)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


In [4]:
df

,ReadingDateTime,SensorGLU
0,2000-02-10 14:21:00,67.0
1,2000-02-10 14:26:00,76.0
2,2000-02-10 14:31:00,86.0
3,2000-02-10 14:36:00,98.0
4,2000-02-10 14:41:00,102.0
...,...,...
343,2000-02-11 18:56:00,58.0
344,2000-02-11 19:01:00,54.0
345,2000-02-11 19:06:00,52.0
346,2000-02-11 19:11:00,55.0


In [5]:
print(vertical_MAE)

[ 4.21845845  3.12574051  4.96558339  8.03545026  7.91580256  7.72935092
  7.37418098  2.85204096  5.18269043  4.18057944  4.19014025  5.90654273
  5.91483585  3.61566361  3.30585783  6.0514585   3.58953026 11.3654356
  8.34585559  5.68393948  4.61736081  5.25357164  3.68607531  5.69821003
  6.75359985  7.27228438  6.21939727  6.81466532  5.89292605  3.69511816
  4.19179727  4.54538872  4.33782216  2.27357104  5.41251095  5.75113058
  5.35544781  5.80581647  6.42421124]


In [6]:
np.median(vertical_MAE)

5.412510946871071

In [7]:
len(vertical_MAE)

39

In [8]:
vertical_RMSE

array([ 6.22004924,  4.34322596,  6.44103686, 11.50791728, 10.3131456 ,
        9.87292017,  9.24507422,  3.95092488,  7.46006337,  5.14663219,
        6.33224619,  7.62934896,  8.14287921,  4.68876549,  4.18622941,
        7.89067898,  4.91481911, 15.52636858, 11.07638945,  7.58146935,
        6.06133896,  6.99209357,  5.54304174,  7.38068266, 10.14161769,
        9.18979453,  9.32103575,  8.6282532 ,  9.63202161,  4.95521809,
        5.9636108 ,  6.34762472,  5.92624194,  3.26881469,  7.14253812,
        7.91487453,  7.15658169,  7.41976006,  8.53863785])

In [9]:
np.median(vertical_RMSE)

7.380682662571159

In [10]:
model.score(X_test,y_test)

0.9795833451891643

In [11]:
import csv


test = pd.DataFrame({"MAE": vertical_MAE, "RMSE": vertical_RMSE, "MMRE": vertical_MMRE, "PRED": vertical_PRED})
test.to_csv(r'/content/Patient_result_SVR_5_part1.csv', index=False)